# Algorithme de Kosaraju et 2-SAT

!jupyterlink

## Kosaraju

On va considérer un type `graphe` où les sommets ne sont pas forcément
des entiers et où on peut stocke une table de hachage pour faire la
correspondance entre la valeur d'un sommet et son indice.

In [ ]:
type 'a graphe = {
    sommets : 'a array;
    sommets_indices : ('a, int) Hashtbl.t;
    ladj : int list array
}

**Question 1**

Écrire une fonction qui étant donné un tableau `t` va construire la
table des associations `t.(i) -> i` qui permettra ainsi de remonter du
tableau à l'indice.

**Note** C'est le moment de retourner apprendre par coeur la doc de
OCaml https://v2.ocaml.org/api/Hashtbl.html

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let table_indices (t:'a array) : ('a, int) Hashtbl.t =
    let h = Hashtbl.create (Array.length t) in
    for i = 0 to Array.length t - 1 do
        Hashtbl.add h t.(i) i
    done;
    h

</div> </div>

**Question 2**

Écrire une fonction `indice` qui prend un graphe et un sommet et renvoie
son indice en utilisant la table `sommets_indices`.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let indice g a = Hashtbl.find g.sommets_indices a

</div> </div>

**Question 3**

En déduire une fonction `cree_graphe` qui va prendre un tableau
`sommets` et créer un graphe sans arêtes dont ce sont les sommets.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let cree_graphe sommets =
    {
        sommets = sommets;
        sommets_indices = table_indices sommets;
        ladj = Array.make (Array.length sommets) []
    }

</div> </div>

**Question 4**

Écrire une fonction `ajoute_arete` telle que `ajoute_arete g a b` ajoute
l'arête `a -> b` où les sommets **sont donnés par leur valeur et non pas
par leur indice**.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let ajoute_arete g a b =
    let ia = indice g a in
    let ib = indice g b in
    if not (List.mem ib g.ladj.(ia))
    then g.ladj.(ia) <- ib :: g.ladj.(ia)

</div> </div>

Les deux cellules suivantes permettent de définir le graphe vu dans le
cours en exemple.

In [ ]:
let ex_cours = cree_graphe [|'a';'b';'c';'d';'e';'f';'g';'h';'i';'j';'k';'l';'m';'n';'o';'p'|]

In [ ]:
List.iter (fun (a,b) -> ajoute_arete ex_cours a b)
    [('a','b'); ('b','f'); ('g','a');('f','g');
     ('c','h'); ('g','c'); ('d','c');('h','d');
     ('e','f'); ('f','l'); ('e','i'); ('g','k');
     ('h','l'); ('i','n'); ('j','m'); ('j','k');
     ('k','l'); ('k','h'); ('l','o'); ('l','p');
     ('m','i'); ('n','j'); ('n','o'); ('o','k')]

**Question 5**

Écrire une fonction `graphe_miroir` qui renvoie le graphe miroir du
graphe donné, c’est-à-dire qui renverse toutes les arêtes.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let graphe_miroir g =
    let gr = {
        sommets = g.sommets;
        sommets_indices = g.sommets_indices;
        ladj = Array.make (Array.length g.sommets) []
    } in
    for i = 0 to Array.length g.sommets - 1 do
        List.iter (fun j -> gr.ladj.(j) <- i :: gr.ladj.(j)) g.ladj.(i)
    done;
    gr

</div> </div>

**Question 6**

Écrire une fonction `dfs` qui va parcourir en profondeur un graphe et
appliquer une fonction de **post-traitement**. Pas besoin de noter les
temps de sortie puisqu’ils correspondent au moment où on effectue ce
traitement.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let rec dfs g trait visites x =
    if not visites.(x)
    then begin
        visites.(x) <- true;
        List.iter (fun y ->
            if not visites.(y)
            then dfs g trait visites y
        ) g.ladj.(x);
        trait x
    end

</div> </div>

**Question 7**

En déduire une fonction `kosaraju` qui renvoie un tableau indiquant le
numéro de la composante fortement connexe de chaque sommet.

Attention, cette fonction va être un peu compliquée, on indique les
grandes lignes ici :

-   on initialise un tableau de visites
-   on va construire une liste `ordre` des indices de sommets rencontrés
    lors d'un premier DFS de tout le graphe grâce à un post-traitement
    (avec une référence)
-   on calcule $G^R$ et on initialise un tableau de visites
-   on fait des DFS dans $G^R$ mais en prenant les sommets depuis la
    liste `ordre`. En post-traitement, on va remplir un tableau `comp`
    où `comp.(x)` sera le numéro de la composante connexe de `x`.

On commence par numéroter les composantes à 1.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let kosaraju g =
    let n = Array.length g.sommets in
    let ordre = ref [] in
    let visites = Array.make n false in
    for i = 0 to n-1 do
        if not visites.(i)
        then dfs g (fun x -> ordre := x :: !ordre) visites i
    done;
    let gr = graphe_miroir g in
    let visites = Array.make n false in
    let ncomp = ref 0 in
    let comp = Array.make n 0 in
    List.iter (fun ia ->
        if not visites.(ia)
        then begin
            incr ncomp;
            dfs gr (fun x -> comp.(x) <- !ncomp) visites ia
        end
        ) !ordre;
    comp

</div> </div>

In [ ]:
assert (kosaraju ex_cours = [|3; 3; 4; 4; 1; 3; 3; 4; 2; 2; 4; 4; 2; 2; 4; 5|]);
"OK"

**Question 8**

Cette représentation des composantes va nous être utile mais elle n’est
pas très maniable. Écrire une fonction `listes_comp` telle que
`listes_comp sommets comp` renvoie le tableau des composantes données
sous forme de listes de sommets.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let listes_comp sommets comp =
    let n = Array.fold_left max 0 comp in
    let comp_l = Array.make n [] in
    for i = 0 to Array.length comp - 1 do
        let n = comp.(i) in
        comp_l.(n-1) <- sommets.(i) :: comp_l.(n-1)
    done;
    comp_l

</div> </div>

In [ ]:
assert (listes_comp ex_cours.sommets (kosaraju ex_cours) = 
    [|['e']; ['n'; 'm'; 'j'; 'i']; 
      ['g'; 'f'; 'b'; 'a'];  ['o'; 'l'; 'k'; 'h'; 'd'; 'c']; ['p']|]);
"OK"

**Question 9**

On va résoudre 2-SAT en utilisant l’algorithme de Kosaraju. Tout d’abord
on rappelle que les formules de 2-SAT ne comportent que deux littéraux
dans chaque clause.

Exemple :
$f = (a \vee \neg b) \wedge (\neg a \vee b) \wedge (\neg a \vee \neg b) \wedge (a \vee \neg c)$

On reprend un type proche de ce qui a été fait l'an dernier, sauf que
les clauses sont des couples de littéraux.

In [ ]:
type lit = Pos of char | Neg of char
type clause = lit * lit
type formule = clause list

La formule donnée au dessus s'écrit alors :

In [ ]:
let f = [ (Pos 'a', Neg 'b'); (Neg 'a', Pos 'b'); (Neg 'a', Neg 'b'); (Pos 'a', Neg 'c') ]

**Question 10**

Écrire une fonction `label : lit -> char` qui renvoie l'étiquette d'un
littéral et une fonction `neg : lit -> lit` qui renvoie le littéral
opposé d'un littéral donné.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let label = function Pos a | Neg a -> a

In [ ]:
let neg = function Pos a -> Neg a | Neg a -> Pos a

</div> </div>

**Question 11**

Écrire une fonction `variables` qui renvoie les étiquettes de tous les
littéraux **sans répétitions**. On pourra utiliser
`List.sort_uniq Stdlib.compare`.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let rec variables f =
    let rec var_aux f =
        match f with
        | [] -> []
        | c::f' -> var_clause c @ var_aux f'
    and var_clause (l1,l2) = [label l1; label l2]
    in List.sort_uniq Stdlib.compare (var_aux f)

</div> </div>

**Question 12**

On peut écrire $a \vee b$ sous la forme implicative
$(\neg a \rightarrow b) \wedge(\neg b \rightarrow a)$.

En faisant cela, on peut alors construire un graphe sur les littéraux où
$l \rightarrow l'$ quand cette implication apparait dans la formule.

Écrire une fonction `implication_graphe` qui étant donnée `f` renvoie ce
graphe.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let implication_graphe f =
    let v = variables f in
    let s = List.concat (List.map (fun x -> [Pos x; Neg x]) v) in
    let sommets = Array.of_list s in
    let g = cree_graphe sommets in
    List.iter (fun (a,b) ->
        ajoute_arete g (neg a) b;
        ajoute_arete g (neg b) a
    ) f;
    g

</div> </div>

On cherche alors une valuation des variables qui soit telle qu'il n'y
ait pas une arête $\top \rightarrow \perp$ entre un littéral vrai et un
littéral faux. En effet, toutes les autres affectations vérifieront la
clause correspondante. Ainsi, tous les littéraux d'une composante
fortement connexe doivent nécessairement avoir la même valeur de vérité.
Si $C \rightarrow C'$ pour deux composantes, on ne peut pas affecter la
valeur vraie à $C$ et fausse à $C'$. On remarque que le graphe est
symétrique par contraposition, donc il existe une composante niée
$\neg C$ pour chaque composante $C$.

L'idée pour résoudre 2-SAT est donc de calculer les compoantes fortement
connexes et de remonter leur DAG en affectant vraie à la première
composante rencontrée entre $C$ et $\neg C$.

Pour résoudre 2-SAT, on va appliquer Kosaraju sur le graphe
d'implication de la formule.

In [ ]:
kosaraju (implication_graphe f)

Sur ce graphe on remarque que si deux littéraux opposés sont dans la
même composante fortement connexe, alors la formule est insoluble. Si ce
n'est pas le cas, on peut démontrer que la valuation suivante fonctionne
:

$$\forall a \in V, v(a) = \begin{cases} \top & \text{si } comp(a) > comp(\neg a) \\
\perp & \text{sinon }\end{cases}$$

Cette affirmation repose sur deux propriétés :

-   les composantes sont numérotées dans l'ordre croissant du tri
    topologique de leur DAG (graphe acyclique orienté)
-   le graphe d'implication possède une symétrie par contraposition : si
    $a_1 \rightarrow a_2 \dots \rightarrow a_n$ alors
    $\neg a_n \rightarrow \neg a_{n-1} \dots \rightarrow \neg a_1$.

**Question 13**

En déduire une fonction `resout_2sat` qui étant donné une formule
renvoie `None` si elle est insoluble, et `Some l` où `l` est une liste
de couples `(variable,valeur)` représentant une valuation solution.

<div class="ui styled accordion"> <div class="title">     <i class="dropdown icon"></i>
Réponse </div> <div class="content">

In [ ]:
let resout_2sat f =
    let g = implication_graphe f in
    let comp = kosaraju g in
    let n = Array.length g.sommets / 2 in
    let valuation = ref [] in
    let insoluble = ref false in
    for i = 0 to n-1 do
        let a = label g.sommets.(2*i) in
        if comp.(2*i) = comp.(2*i+1)
        then insoluble := true;
    done;
    if !insoluble
    then None
    else Some !valuation

</div> </div>

In [ ]:
resout_2sat f

!reponses